In [52]:
import xgboost as xgb
import alpaca_trade_api as tradeapi
from finta import TA
import pickle

# Initial imports
import os
import requests
import pandas as pd
from dotenv import load_dotenv

In [53]:
papertrading_model = xgb.XGBClassifier()
papertrading_model.load_model('xgb_clf.bst')
scaler = pickle.load(open('scaler_model.pkl','rb'))



/Users/adriencaudron/miniforge3/envs/dev310/lib/python3.10/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator StandardScaler from version 1.2.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [54]:
feats = ['HMA_5', 'RSI_5', 'ATR_14', 'RSI_14', 'RSI_150', 'cci']

In [55]:
alpaca_api = tradeapi.REST(alpaca_api_key,alpaca_secret_key, 'https://paper-api.alpaca.markets')

In [56]:
data_trade = alpaca_api.get_crypto_bars(['BTC/USDT'], tradeapi.TimeFrame.Hour, "2023-01-01").df

In [57]:
for time in [5,14,150]:
    data_trade['RSI_'+str(time)] = TA.RSI(data_trade, time)
data_trade['ATR_14'] = TA.ATR(data_trade, 14)
data_trade['HMA_5'] = TA.HMA(data_trade, 5)
data_trade['cci'] = TA.CCI(data_trade)

In [58]:
data_trade

,close,high,low,trade_count,open,volume,vwap,symbol,RSI_5,RSI_14,RSI_150,ATR_14,HMA_5,cci
timestamp,,,,,,,,,,,,,,
2023-01-01 00:00:00+00:00,16526.09,16533.00,16500.63,469,16524.41,16.810355,16519.905696,BTC/USDT,NaN,NaN,NaN,NaN,NaN,NaN
2023-01-01 01:00:00+00:00,16546.99,16546.99,16518.79,421,16528.29,11.604272,16530.709062,BTC/USDT,100.000000,100.000000,100.000000,NaN,NaN,66.666667
2023-01-01 02:00:00+00:00,16539.38,16554.60,16529.80,243,16537.70,9.440864,16539.153275,BTC/USDT,68.721743,71.832699,73.176520,NaN,NaN,64.102126
2023-01-01 03:00:00+00:00,16521.36,16537.40,16508.76,322,16536.57,12.603051,16527.232191,BTC/USDT,35.684559,41.804749,44.635454,NaN,NaN,-57.152441
2023-01-01 04:00:00+00:00,16520.57,16533.60,16514.69,245,16530.82,6.139097,16524.210877,BTC/USDT,34.768595,40.995657,43.880112,NaN,NaN,-46.385258
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-03-13 20:00:00+00:00,24136.06,24311.82,24063.84,2593,24273.31,191.102030,24174.956257,BTC/USDT,79.026858,79.926357,57.942146,424.510000,24244.798222,113.115714
2023-03-13 21:00:00+00:00,24081.79,24213.42,24043.17,1181,24137.99,53.203990,24135.518584,BTC/USDT,73.636229,78.201474,57.723558,423.792143,24087.176222,93.901846
2023-03-13 22:00:00+00:00,24199.82,24232.95,24079.65,1555,24079.65,108.127990,24175.819906,BTC/USDT,77.760381,79.250289,58.069892,409.147143,24113.632667,87.032091


In [59]:
data_to_predict = scaler.transform(data_trade[feats].dropna())

In [60]:
data_to_predict[-1]

array([91.86807861,  0.38909019, 39.86967123,  1.89722956,  1.75711158,
        0.55700817])

In [64]:
## TEST to place a buy order for 500 USD worth of ETH using Alpaca API in Python:

# Define order parameters
symbol = 'ETHUSD'
order_type = 'limit'
limit_price = '1400' # limit price for the buy order
time_in_force = 'gtc'
qty = '0.25' # quantity of ETH to buy
side = 'buy'

# Place buy order
try:
    order = alpaca_api.submit_order(
        symbol=symbol,
        qty=qty,
        side=side,
        type=order_type,
        time_in_force=time_in_force,
        limit_price=limit_price
    )
    print(f"Buy order for {qty} {symbol} at {limit_price} submitted successfully.")
except Exception as e:
    print("Error submitting buy order: ", e)


Buy order for 0.25 ETHUSD at 1400 submitted successfully.


In [61]:
quantity_to_buy = float(alpaca_api.get_position('USDTUSD').qty)/float(alpaca_api.get_latest_crypto_orderbook(['BTC/USDT'])['BTC/USDT'].bids[0].p)


APIError: position does not exist

In [ ]:
alpaca_api.get_latest_crypto_orderbook(['BTC/USDT'])['BTC/USDT'].bids[0].p

23995.808802

In [ ]:
quantity_to_buy

NameError: name 'quantity_to_buy' is not defined

In [ ]:
def trade_in_alpaca(quantity, tries=0):
    if papertrading_model.predict(data_to_predict)[-1]==1 and papertrading_model.predict(data_to_predict)[-2]==0:
        try:
            alpaca_api.submit_order(symbol='BTC/USDT',qty=0.04,side="buy",type='market', time_in_force='ioc')
        except Exception as e:
            if 'insufficient balance' in str(e) and tries<=10:
                print(quantity, tries)
                trade_in_alpaca(quantity=0.9*quantity, tries=tries+1)
            else:
                print(e)

In [ ]:
trade_in_alpaca(quantity=quantity_to_buy)

NameError: name 'quantity_to_buy' is not defined

In [ ]:
alpaca_api.list_positions()

[Position({   'asset_class': 'us_equity',
     'asset_id': 'b0b6dd9d-8b9b-48a9-ba46-b9d54906e415',
     'asset_marginable': False,
     'avg_entry_price': '125.25',
     'change_today': '0.0161616161616162',
     'cost_basis': '125.25',
     'current_price': '150.9',
     'exchange': 'NASDAQ',
     'lastday_price': '148.5',
     'market_value': '150.9',
     'qty': '1',
     'qty_available': '1',
     'side': 'long',
     'symbol': 'AAPL',
     'unrealized_intraday_pl': '2.4',
     'unrealized_intraday_plpc': '0.0161616161616162',
     'unrealized_pl': '25.65',
     'unrealized_plpc': '0.2047904191616766'}),
 Position({   'asset_class': 'us_equity',
     'asset_id': 'f30d734c-2806-4d0d-b145-f9fade61432b',
     'asset_marginable': False,
     'avg_entry_price': '111.43',
     'change_today': '0.0107680474673113',
     'cost_basis': '2228.6',
     'current_price': '91.99',
     'exchange': 'NASDAQ',
     'lastday_price': '91.01',
     'market_value': '1839.8',
     'qty': '20',
     'qty